Лабораторна Робота №7.


Задача голосування порядку денного 
(ранжування визначає значущість проекту та пріоритетність у фінансуванні) Список проектів-кандидатів: 
A. відновлення смертної кари 
B. легалізація наркотиків 
C. заборона алкогольної реклами на ТВ 
D. реєстрація гомосексуальних шлюбів 

Варіант № 7.
  
2	8	4	6 <br>
C	B	A	C <br>
A	C	B	A <br>
B	A	C	D <br>
D	D	D	B <br>

In [339]:
from collections import defaultdict
import math

votes = [
   ("C", "A", "B", "D", 2),
   ("B", "C", "A", "D", 8),
   ("A", "B", "C", "D", 4),
   ("C", "A", "D", "B", 6),
]


### Абсолютна більшість

In [340]:
def absolute_majority(votes):
   scores = defaultdict(int)
   
   for vote in votes:
      first_choice = vote[0]
      scores[first_choice] += vote[-1]

   half_candidates = math.ceil(len(scores) / 2)
   best_candidates = dict(sorted(scores.items(), key=lambda x: x[1], reverse=True)[:half_candidates])

   if len(best_candidates) > 1:
      remaining_votes = []
      for vote in votes:
            remaining_vote = tuple(c for c in vote[:-1] if c in best_candidates)
            if remaining_vote:
               remaining_votes.append(remaining_vote + (vote[-1],))
      absolute_majority(remaining_votes)
   else:
      winner = list(best_candidates.keys())[0]
      print(f"Правило абсолютної більшості: Переможець {winner} з {scores[winner]} голосами")


absolute_majority(votes)


Правило абсолютної більшості: Переможець B з 12 голосами


### Відносна більшість

In [341]:
def relative_majority(votes):
    scores = defaultdict(int)

    for vote in votes:
        first_choice = vote[0]
        scores[first_choice] += vote[-1]

    winner = max(scores, key=scores.get)

    print(f"Правило абсолютної більшості: Переможець {winner} з {scores[winner]} голосами")


relative_majority(votes)

Правило абсолютної більшості: Переможець C з 8 голосами


### Правило Борда

In [342]:
def bord_rule(votes):
    candidate_scores = defaultdict(int)

    for vote in votes:
        for i, candidate in enumerate(vote[:-1]):
            candidate_scores[candidate] += vote[-1] * (3 - i)

    sorted_candidates = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)

    winner, winning_score = sorted_candidates[0]
    print(f"Переможець методом Борда: {winner} з {winning_score} балами.")


bord_rule(votes)


Переможець методом Борда: C з 44 балами.


### Правило Кондорсе

In [343]:
def condorcet_rule(votes):
    candidates = set(candidate for vote in votes for candidate in vote[:-1])

    pairwise_wins = {pair: 0 for pair in [(c1, c2) for c1 in candidates for c2 in candidates if c1 != c2]}

    for vote in votes:
        for i, candidate1 in enumerate(vote[:-1]):
            for j in range(i + 1, len(vote) - 1):
                candidate2 = vote[j]
                pairwise_wins[(candidate1, candidate2)] += vote[-1]

    condorcet_winner = max(candidates, key=lambda candidate: sum(pairwise_wins.get((candidate, other), 0) > pairwise_wins.get((other, candidate), 0) for other in candidates))

    print(f"Переможець за правилом Кондорсе: {condorcet_winner}")

    for other in candidates:
        if condorcet_winner != other:
            wins = pairwise_wins.get((condorcet_winner, other), 0)
            loses = pairwise_wins.get((other, condorcet_winner), 0)
            print(f"{condorcet_winner} порівняно з {other} {wins} : {loses}")


condorcet_rule(votes)

Переможець за правилом Кондорсе: B
B порівняно з A 8 : 12
B порівняно з D 14 : 6
B порівняно з C 12 : 8


### Правило Копленда

In [344]:
def calculate_pairwise_votes(votes):
    candidates = set(candidate for vote in votes for candidate in vote[:-1])

    pairwise_votes = {pair: 0 for pair in [(c1, c2) for c1 in candidates for c2 in candidates if c1 != c2]}

    for vote in votes:
        for i, candidate1 in enumerate(vote[:-1]):
            for j in range(i + 1, len(vote) - 1):
                candidate2 = vote[j]
                pairwise_votes[(candidate1, candidate2)] += vote[-1]

    return pairwise_votes

def copeland_rule(votes):
    candidates = set(candidate for vote in votes for candidate in vote[:-1])

    pairwise_votes = calculate_pairwise_votes(votes)

    candidate_scores = {candidate: 0 for candidate in candidates}

    for candidate_a in candidates:
        for candidate_x in candidates:
            if candidate_a != candidate_x:
                if pairwise_votes[(candidate_a, candidate_x)] > pairwise_votes[(candidate_x, candidate_a)]:
                    candidate_scores[candidate_a] += 1
                elif pairwise_votes[(candidate_a, candidate_x)] < pairwise_votes[(candidate_x, candidate_a)]:
                    candidate_scores[candidate_a] -= 1
                    
    winner = max(candidate_scores, key=candidate_scores.get)

    print(f"Переможець за правилом Копленда {winner} з {candidate_scores[winner]} балами.")


copeland_rule(votes)

Переможець за правилом Копленда B з 1 балами.


### Правило Сімпсона

In [345]:
def simpson_rule(votes):
    candidates = set()
    pairwise_sums = {}

    for vote in votes:
        for candidate in vote[:-1]:
            candidates.add(candidate)

    for candidate1 in candidates:
        for candidate2 in candidates:
            if candidate1 != candidate2:
                pairwise_sums[(candidate1, candidate2)] = 0

    for vote in votes:
        for i in range(len(vote) - 1):
            for j in range(i + 1, len(vote) - 1):
                candidate1 = vote[i]
                candidate2 = vote[j]
                if candidate1 != candidate2:
                    pairwise_sums[(candidate1, candidate2)] += vote[-1]

    candidate_min_max_sums = {}
    for candidate in candidates:
        min_sum = min(pairwise_sums[(candidate, other)] for other in candidates if candidate != other)
        candidate_min_max_sums[candidate] = min_sum

    winner = max(candidate_min_max_sums, key=candidate_min_max_sums.get)

    print(f"Переможець за правилом Сімпсона {winner} з {candidate_min_max_sums[winner]} балами.")


simpson_rule(votes)

Переможець за правилом Сімпсона B з 8 балами.
